In [1]:
import numpy as np

In [4]:
def _union(windows):
    """
    modifed from https://stackoverflow.com/questions/15273693/union-of-multiple-ranges

    return a list of windows that is the union of the input
    
    input windows: [[7.1, 11.5, [0.32]], [11.0, 13.0, [0.41]], [11.0, 14.9, [0.32]], [15.0, 20.0, [0.27, 0.35]], [23.0, 39.0, [0.40, 0.44]]]
    input doesn't need to be sorted

    return: [[7.1, 14.9, [0.32, 0.41, 0.32]], [15.0, 20.0, [0.27, 0.35]], [23., 39., [0.40, 0.44]]]
    the similarity score is just the mean of combined segments
    """
    b = []
    for begin, end, score in sorted(windows):
        if b and b[-1][1] >= begin:
            b[-1][1] = max(b[-1][1], end)
            b[-1][2] += score
        else:
            b.append([begin, end, score])
    return b

def temporal_nms(bboxes, thresh, score_ind=2):
    """
    One-dimensional non-maximal suppression
    :param bboxes: [[st, ed, score], ...]
    :param thresh:
    :return:


    copied (and slightly modified) from https://github.com/yjxiong/action-detection/blob/3d8ff836387fb977979ef792152a6b6ac43725fc/ops/sequence_funcs.py#L71
    """
    t1 = np.array([x[0] for x in bboxes])
    t2 = np.array([x[1] for x in bboxes])
    scores = np.array([x[score_ind] for x in bboxes])

    durations = t2 - t1 + 1
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        tt1 = np.maximum(t1[i], t1[order[1:]])
        tt2 = np.minimum(t2[i], t2[order[1:]])
        intersection = tt2 - tt1 + 1
        IoU = intersection / (durations[i] + durations[order[1:]] - intersection).astype(float)

        inds = np.where(IoU <= thresh)[0]
        order = order[inds + 1]

    return [bboxes[i] for i in keep]


def watershed_single(windows, gamma, tau):
    """
    run 1D watershed algo on predicted windows
    method introduced in https://arxiv.org/pdf/1704.06228.pdf
    
    input:
        gamma: similarity threshold
        tau: window_dur/total_dur threshold
        windows: [[1.2, 15.3, 0.32], [15.3, 18.6, 0.34], [18.6, 23.7, 0.12], [23.7, 24.6, 0.44], [25.7, 28.4, 0.32], [30.1, 34.5, 0.45], [34.5, 35.6, 0.01], [35.6, 38.4, 0.34]]
            segments of [start_time, end_time, similarity_score]
            the windows can overlap
            windows should be sorted based on start_time

        tau and gamma are floating point between 0 and 1
    
    output:
    (suppose gamma = 0.32 and tau = 0.85)
        merged_windows: [[1.2, 18.6, [0.32, 0.34]],
                    [23.7, 24.6, [0.44]],
                    [25.7, 28.4, [0.32]],
                    [30.1, 38.4, [0.45, 0.34]]]
        merged segments
    """
    scores = [item[2] for item in windows]
    remain_windows = np.array(windows)[np.where(np.array(scores) >= gamma)[0]]
    if len(remain_windows) == 0:
        return None
    remain_windows = [[w[0], w[1], [w[2]]] for w in remain_windows] # [[1.2, 15.3, 0.32], [15.3, 18.6, 0.34], ....] -> [[1.2, 15.3, [0.32]], [15.3, 18.6, [0.34]], ....]
    groups = []
    for i, cur_win in enumerate(remain_windows):
        window_union = _union(groups[-1]+[cur_win]) if groups else [cur_win]
        window_duration = sum([item[1] - item[0] for item in window_union])
        total_duration = window_union[-1][1] - window_union[0][0]
        if window_duration/total_duration <= tau or i == 0:
            groups.append([cur_win])
        else:
            groups[-1] = window_union
        merged_groups = [[cur_group[0][0],cur_group[-1][1], [ss for s in cur_group for ss in s[2]]] for cur_group in groups]
    return merged_groups


In [14]:
windows = [[1.2, 15.3, 0.32], [15.3, 18.6, 0.34], [18.6, 23.7, 0.12], [23.7, 24.6, 0.44], [25.7, 28.4, 0.32], [30.1, 34.5, 0.45], [34.5, 35.6, 0.01], [35.6, 38.4, 0.34]]
gamma = 0.32
tau = 0.85
scores = [item[2] for item in windows]
remain_windows = np.array(windows)[np.where(np.array(scores) >= gamma)[0]]
remain_windows = [[w[0], w[1], [w[2]]] for w in remain_windows] # [[1.2, 15.3, 0.32], [15.3, 18.6, 0.34], ....] -> [[1.2, 15.3, [0.32]], [15.3, 18.6, [0.34]], ....] 
groups = []
for i, cur_win in enumerate(remain_windows):
    window_union = _union(groups[-1]+[cur_win]) if groups else [cur_win]
    window_duration = sum([item[1] - item[0] for item in window_union])
    total_duration = window_union[-1][1] - window_union[0][0]
    if window_duration/total_duration <= tau or i == 0:
        groups.append([cur_win])
        print(groups)
    else:
        groups[-1] = window_union
        print(groups)
    merged_groups = [[cur_group[0][0],cur_group[-1][1], [ss for s in cur_group for ss in s[2]]] for cur_group in groups]

[[[1.2, 15.3, [0.32]]]]
[[[1.2, 18.6, [0.32, 0.34]]]]
[[[1.2, 18.6, [0.32, 0.34]]], [[23.7, 24.6, [0.44]]]]
[[[1.2, 18.6, [0.32, 0.34]]], [[23.7, 24.6, [0.44]]], [[25.7, 28.4, [0.32]]]]
[[[1.2, 18.6, [0.32, 0.34]]], [[23.7, 24.6, [0.44]]], [[25.7, 28.4, [0.32]]], [[30.1, 34.5, [0.45]]]]
[[[1.2, 18.6, [0.32, 0.34]]], [[23.7, 24.6, [0.44]]], [[25.7, 28.4, [0.32]]], [[30.1, 34.5, [0.45]], [35.6, 38.4, [0.34]]]]


In [15]:
merged_groups

[[1.2, 18.6, [0.32, 0.34]],
 [23.7, 24.6, [0.44]],
 [25.7, 28.4, [0.32]],
 [30.1, 38.4, [0.45, 0.34]]]

In [19]:
all_proposals = []
gammas = [0.32, 0.35]
taus = [0.85, 0.9]
iou_threshold = 0.8
for gamma in gammas:
    for tau in taus:
        all_proposals += watershed_single(windows, gamma, tau)
        print(all_proposals)
proposals = [[item[0], item[1], np.mean(item[2])] for item in _union(all_proposals)]
proposals = temporal_nms(proposals, thresh = iou_threshold)


[[1.2, 18.6, [0.32, 0.34]], [23.7, 24.6, [0.44]], [25.7, 28.4, [0.32]], [30.1, 38.4, [0.45, 0.34]]]
[[1.2, 18.6, [0.32, 0.34]], [23.7, 24.6, [0.44]], [25.7, 28.4, [0.32]], [30.1, 38.4, [0.45, 0.34]], [1.2, 18.6, [0.32, 0.34]], [23.7, 24.6, [0.44]], [25.7, 28.4, [0.32]], [30.1, 34.5, [0.45]], [35.6, 38.4, [0.34]]]
[[1.2, 18.6, [0.32, 0.34]], [23.7, 24.6, [0.44]], [25.7, 28.4, [0.32]], [30.1, 38.4, [0.45, 0.34]], [1.2, 18.6, [0.32, 0.34]], [23.7, 24.6, [0.44]], [25.7, 28.4, [0.32]], [30.1, 34.5, [0.45]], [35.6, 38.4, [0.34]], [23.7, 24.6, [0.44]], [30.1, 34.5, [0.45]]]
[[1.2, 18.6, [0.32, 0.34]], [23.7, 24.6, [0.44]], [25.7, 28.4, [0.32]], [30.1, 38.4, [0.45, 0.34]], [1.2, 18.6, [0.32, 0.34]], [23.7, 24.6, [0.44]], [25.7, 28.4, [0.32]], [30.1, 34.5, [0.45]], [35.6, 38.4, [0.34]], [23.7, 24.6, [0.44]], [30.1, 34.5, [0.45]], [23.7, 24.6, [0.44]], [30.1, 34.5, [0.45]]]


In [20]:
proposals

[[23.7, 24.6, 0.44],
 [30.1, 38.4, 0.41333333333333333],
 [1.2, 18.6, 0.33],
 [25.7, 28.4, 0.32]]

In [6]:
windows = [[0.0, 3.267, 0.1974087357521057], [3.267, 31.667, 0.25133728981018066], [31.667, 36.7, 0.2245093584060669], [36.7, 43.6, 0.2003909945487976], [43.6, 50.2, 0.22681182622909546], [50.2, 51.9, 0.22681182622909546], [51.9, 59.233, 0.29440170526504517], [59.233, 78.766, 0.2975807785987854], [78.766, 98.3, 0.30237793922424316], [98.3, 104.367, 0.22737812995910645], [104.367, 107.933, 0.20067095756530762], [107.933, 113.133, 0.20931106805801392], [113.133, 123.833, 0.20816028118133545], [123.833, 129.5, 0.2564511299133301], [129.5, 142.333, 0.279940664768219], [142.333, 150.0, 0.21298527717590332]]
all_proposals = []
gammas = [0.24, 0.25, 0.30, 0.32, 0.34]
taus = [0.85, 0.9]
iou_threshold = 0.8
for gamma in gammas:
    for tau in taus:
        cur_proposal =  watershed_single(windows, gamma, tau)
        if cur_proposal != None:
            all_proposals += cur_proposal
        # print(all_proposals)
proposals = [[item[0], item[1], np.mean(item[2])] for item in _union(all_proposals)]
proposals = temporal_nms(proposals, thresh = iou_threshold)

In [7]:
proposals

[[51.9, 98.3, 0.2987283979143415],
 [123.833, 142.333, 0.26819589734077454],
 [3.267, 31.667, 0.25133728981018066]]